## **0. 합성곱과 풀링**

**합성곱 신경망(Convolutional Neural Network)** 은 이미지 처리에 탁월한 성능을 보이는 신경망입니다. 합성곱 신경망은 크게 **합성곱층(Convolution layer)** 과 **풀링층(Pooling layer)** 으로 구성됩니다. 아래의 그림은 합성곱 신경망의 일반적인 예를 보여줍니다.

<img src = "https://wikidocs.net/images/page/62306/convpooling.PNG">

위 그림에서 CONV는 합성곱 연산을 의미하고, 합성곱 연산의 결과가 활성화 함수 ReLU를 지납니다. 이 두 과정을 합성곱층이라고 합니다. 그 후에 POOL이라는 구간을 지나는데 이는 풀링 연산을 의미하여 풀링층이라고 합니다. 

## **1. 합성곱 신경망의 대두**

이미지 처리를 하기 위해서 앞서 배운 다층 퍼셉트론을 사용할 수는 있지만 한계가 있었습니다. 예를 들어, 알파벳 손글씨를 분류하는 어떤 문제가 있습니다. 알파벳 y를 손글씨로 쓴 두 가지 예시를 행렬로 표현한 것이 다음과 같습니다.

<img src = "https://wikidocs.net/images/page/64066/conv0.png">

사람이 보기에 두 그림은 모두 y로 보이지만 기계가 보기에 각 픽셀마다 가진 값이 거의 상이하므로 완전히 다른 입력으로 받아들입니다. 그리고 두 이미지 외에도 휘어지거나, 이동되거나 방향이 뒤틀리는 등 다양한 변형이 존재합니다. 다층 퍼셉트론은 몇 가지 픽셀만 값이 달라져도 민감하게 받아들이기에 적합하지 않습니다. 

이를 더 자세히 살펴봅시다. 만약 위 이미지를 다층 퍼셉트론으로 분류한다면 이미지를 1차원 텐서인 벡터로 변환하여 입력층으로 사용해야 합니다. 이를 전환하면 다음과 같습니다.

<img src = "https://wikidocs.net/images/page/64066/conv1.png">

1차원으로 변환된 결과는 사람이 보기에 원래 이미지를 유추하기 매우 어렵습니다. 이는 기계 역시 마찬가집니다. 위와 같은 변환은 전에 가지고 있던 공간적인 구조(spatial structure) 정보가 유실된 상태입니다. 그렇기에 다층 퍼셉트론으로 이미지를 분류하기엔 어렵습니다. 그렇기에 이미지의 공간적인 구조 정보를 보존하면서 학습할 수 있는 방법이 필요해졌고 이를 위해 사용하는 것이 합성곱 신경망입니다.

## **2. 채널(Channel)**

이미지 처리의 기본적인 용어인 채널에 대해서 간단히 정의하겠습니다.

기계는 글자나 이미지보다, 텐서를 더 잘 처리할 수 있습니다. 이미지는 **(높이, 너비, 채널)**이라는 3차원 텐서입니다. 여기서 높이는 세로 방향 픽셀 수, 너비는 이미지의 가로 방향 픽셀 수, 채널은 색 성분을 의미합니다. 흑백 이미지는 채널 수가 1이며 각 픽셀은 0부터 255 사이의 값을 가집니다. 아래는 28 x 28 픽셀의 손글씨 데이터를 보여줍니다.

<img src = "https://wikidocs.net/images/page/64066/conv2.png">

위 손글씨 데이터는 흑백 이미지이므로 채널 수가 1입니다. 그렇기에 위 이미지는 (28 x 28 x 1)의 크기를 가지는 3차원 텐서입니다. 만약 컬러 이미지라면 RGB가 각각 채널이 1개씩 총 3개를 가집니다. 

<img src = "https://wikidocs.net/images/page/64066/conv3.png">

하나의 픽셀은 삼원색의 조합으로 이루어집니다. 만약, 높이와 너비가 28인 컬러 이미지가 있다면 이 이미지의 텐서는 (28 x 28 x 3)의 크기를 가지는 3차원 텐서입니다. 채널은 떄로는 깊이(depth)라고도 합니다. 

## **3. 합성곱 연산(Convolution operation)**

합성곱층은 합성곱 연산을 통해서 이미지의 특징을 추출하는 역할을 합니다. 합성곱은 kernel 또는 filter라는 n x m크기의 행렬로 height x width 크기의 이미지를 처음부터 끝까지 겹치고 훓으면서 n x m 크기의 겹쳐지는 부분의 각 이미지와 kernel의 원소의 값을 곱합니다. 그리고 이를 모두 더하여 값으로 출력하는 것을 말합니다. 

- kernel은 일반적으로 3 x 3, 5 x 5를 많이 사용합니다.

예시를 보겠습니다. 아래는 3 x 3크기의 커널로 5 x 5 이미지 행렬에 합성곱 연산을 수행하는 과정을 보여줍니다. 한 번의 연산을 1 step이라고 했을 때, 합성곱 연산의 네번째 스텝까지 이미지와 식으로 보겠습니다. 

1) 첫번째 스텝  

<img src = "https://wikidocs.net/images/page/64066/conv4.png">

$(1 \times 1) + (2 \times 0) + (3 \times 1) + (2 \times 1) + (1 \times 0)+ (0 \times 1)+ (3 \times 0)+ (0 \times 1) + (1 \times 0) = 6$


2) 두번째 스텝  

<img src = "https://wikidocs.net/images/page/64066/conv5.png">

$(2 \times 1) + (3 \times 0) + (4 \times 1) + (1 \times 1) + (0 \times 0)+ (1 \times 1)+ (0 \times 0)+ (1 \times 1) + (1 \times 0) = 9$


3) 세번째 스텝  

<img src = "https://wikidocs.net/images/page/64066/conv6.png">

$(3 \times 1) + (4 \times 0) + (5 \times 1) + (0 \times 1) + (1 \times 0)+ (2 \times 1)+ (1 \times 0)+ (1 \times 1) + (0 \times 0) = 11$


4) 네번째 스텝  

<img src = "https://wikidocs.net/images/page/64066/conv7.png">

$(2 \times 1) + (1 \times 0) + (0 \times 1) + (3 \times 1) + (0 \times 0)+ (1 \times 1)+ (1 \times 0)+ (4 \times 1) + (1 \times 0) = 10$


총 9번의 스텝을 했을 때 최종 결과는 다음과 같습니다.

<img src = "https://wikidocs.net/images/page/64066/conv8.png">

위와 같이 입력으로부터 커널을 사용하여 합성곱 연산을 통해 나온 결과를 **특성 맵(feature map)** 이라고 합니다.

위의 예제에선 커널의 크기 3 x 3이었지만, 커널의 크기는 사용자가 지정할 수 있습니다. 또한 커널의 이동 범위가 위의 예제에서는 한 칸이었지만, 이 또한 사용자가 정할 수 있습니다. 이러한 이동 범위를 **스트라이드(stride)** 라고 합니다.

아래의 예제는 스트라이드가 2일 때, 5 x 5 이미지에 합성곱 연산을 수행하는 3 x 3 커널의 움직임을 보여줍니다. 최종적으로 2 x 2 크기의 특성 맵을 얻습니다.

<img src = "https://wikidocs.net/images/page/64066/conv9.png">

## **4. 패딩(Padding)**

위의 예시처럼 합성곱 연산으로 얻은 특성 맵은 입력보다 크기가 작아지는 특징이 있습니다. 만약, 합성곱 층을 여러개 쌓았다면 최종적으로 얻는 특성 맵의 크기는 입력보다 매우 작아진 상태가 됩니다. 만약 합성곱 연산 이후에도 특성 맵의 크기가 입력 크기와 동일하게 유지되도록 하고 싶다면 패딩(padding)을 사용합니다.

<img src = "https://wikidocs.net/images/page/64066/conv10.png">

패딩은 합성곱 연산을 하기 전에 입력의 가장자리에 지정된 개수의 폭만큼 행과 열을 추가하는 것을 말합니다. 다시 말해 지정된 개수의 폭만큼 테두리를 추가합니다. 주로 0으로 채우는 제로 패딩(zeor padding)을 사용합니다. 위의 그림은 5 x 5 이미지에 1폭짜리 제로 패딩을 사용한 모습입니다.

만약 스트라이드가 1일 때, 3 x 3 크기의 커널을 사용한다면 1폭짜리 제로 패딩을 사용하고, 5 x 5 크기의 커널을 사용한다면 2폭짜리 제로 패딩을 사용하여 크기를 보존할 수 있습니다. 

## **5. 가중치와 편향**

합성곱 신경망에서의 가중치와 편향을 이해하기 위해 먼저 다층 퍼셉트론을 복습하겠습니다.

### 5.1 합성곱 신경망의 가중치

다중 퍼셉트론으로 3 x 3 이미지를 처리한다고 가정하겠습니다. 우선 이미지를 1차원 텐서로 만들면 입력층은 9개의 뉴론을 가집니다. 그리고 4개의 뉴론을 가지는 은닉층을 추가한다면 아래의 그림과 같아집니다.

<img src = "https://wikidocs.net/images/page/64066/conv11.png">

위에서 각 연결선은 가중치를 의미하므로 위의 그림에서는 36(=9 x 4)개의 가중치를 가집니다.

같은 이미지를 합성곱 신경망으로 처리해보겠습니다. 2 x 2 커널을 사용하고 스트라이드는 1로 하겠습니다. (*는 합성곱 연산을 의미합니다.)

<img src = "https://wikidocs.net/images/page/64066/conv12.png">

합성곱 신경망에서 가중치는 커널 행렬의 원소들입니다. 이를 인공 신경망으로 표현하면 다음과 같습니다.

<img src = "https://wikidocs.net/images/page/64066/conv13.png">

최종적으로 특성 맵을 얻기 위해서 동일한 커널로 이미지 전체를 훑으면서 합성곱 연산을 진행합니다. 결국 이미지 전체를 훑으면서 사용되는 가중치는 $w_0, w_1, w_2, w_3$ 4개 뿐입니다. 그리고 각 합성곱 연산마다 이미지의 모든 픽셀을 사용하는 것이 아니라 커널과 맵핑되는 픽셀만을 입력으로 사용하는 것을 볼 수 있습니다. 결국 합성곱 신경망은 다층 퍼셉트론보다 훨씬 적은 가중치를 사용하며 공간적 구조 정보를 보존하는 특징을 가집니다.

다층 퍼셉트론의 은닉층에서 가중치 연산 이후, 비선형성을 위해 활성화 함수를 통과시켰듯, 합성곱 신경망에서도 합성곱 연산을 통해 얻은 특성 맵을 활성화 함수를 통과시켜 비선형성을 갖게 만듭니다. 이때 활성화 함수로 ReLU나 그 변형들이 주로 사용됩니다. 이와 같이 합성곱 연산을 통해 특성 맵을 얻고, 활성화 함수를 지나는 연산을 하는 합성곱 신경망의 은닉층을 **합성곱 층(convolution layer)** 이라고 합니다.

### 5.2 합성곱 신경망의 편향

<img src = "https://wikidocs.net/images/page/64066/conv14.png">

합성곱 신경망에도 편향을 추가할 수 있습니다. 만약, 편향을 사용한다면 커널을 적용한 뒤에 더해집니다. 편향은 하나의 값만 존재하며 커널이 적용된 결과의 모든 원소에 더해집니다.

## **6. 특성 맵의 크기 계산 방법**

합성곱 연산의 notation을 먼저 확인하겠습니다.

- $I_h$: 입력의 높이
- $I_w$: 입력의 너비
- $K_h$: 커널의 높이
- $K_w$: 커널의 너비
- $S$: 스트라이드
- $O_h$: 특성 맵의 높이
- $O_w$: 특성 맵의 너비

이에 따라 특성 맵의 높이와 너비는 다음과 같습니다.

$$O_h = floor(\frac{I_h - K_h}{S} + 1)$$

$$O_w = floor(\frac{I_w - K_w}{S} + 1)$$

여기서 $floor$ 함수는 소수점 발생 시, 소수점 이하를 버리는 역할을 합니다. 

예를 들어 5 x 5 크기의 이미지에 3 x 3 커널을 사용하고 스트라이드 1로 합성곱 연산을 한 경우, 특성 맵의 크기는 (5 - 3 + 1) x (5 - 3 + 1) = 3 x 3임을 알 수 있습니다. 이는 또한 9번의 스텝이 필요함을 의미하기도 합니다.

패딩의 폭을 $P$라고 할=고, 패딩까지 고려한 식은 다음과 같습니다.

$$O_h = floor(\frac{I_h - K_h + 2P}{S} + 1)$$

$$O_w = floor(\frac{I_w - K_w + 2P}{S} + 1)$$

## **7. 다수의 채널을 가질 경우의 합성곱 연산(3차원 텐서의 합성곱 연산)**

지금까지는 채널 또는 깊이를 고려하지 않고, 2차원 텐서를 가정하고 설명했습니다. 하지만 실제로 합성곱 연산의 입력은 '다수의 채널을 가진' 이미지 또는 이전 연산의 결과로 나온 특성 맵일 수 있습니다. 만약, 다수의 채널을 가진 입력 데이터를 가지고 합성곱 연산을 한다고 하면 커널의 채널 수도 입력의 채널 수만큼 존재해야 합니다. 다시 말해 입력 데이터의 채널 수와 커널의 채널 수는 같아야 합니다. 채널 수가 같으므로 합성곱 연산을 채널마다 수행합니다. 그리고 그 결과를 모두 더하여 최종 특성 맵을 얻습니다. 

<img src = "https://wikidocs.net/images/page/64066/conv15.png">

위 그림은 3개의 채널을 가진 입력 데이터와 3개의 채널을 가진 커널의 합성곱 연산을 보여줍니다. 커널의 각 채널끼리의 크기는 같아야 합니다. 각 채널간 합성곱 연산을 마치고, 그 결과를 모두 더해서 하나의 채널을 가지는 특성 맵을 만듭니다. 주의할 점은 위의 연산에서 사용되는 커널은 3개의 커널이 아니라 3개의 채널을 가진 1개의 커널이라는 것입니다.

위 그림은 높이 3, 너비 3, 채널 3의 입력이 높이 2, 너비 2, 채널 3의 커널과 합성곱 연산을 하여 높이 2, 너비 2, 채널 1의 특성 맵을 얻는다는 의미입니다. 합성곱 연산의 결과로 얻은 특성 맵의 태널 차원은 RGB 채널 등과 같은 컬러의 의미를 담고 있지 않습니다.

이제 이 연산에서 각 차원을 변수로 두고 좀 더 일반화시켜보겠습니다.

## **8. 3차원 텐서의 합성곱 연산**

일반화를 위해 사용하는 변수들은 다음과 같습니다.

- $I_h$: 입력의 높이
- $I_w$: 입력의 너비
- $K_h$: 커널의 높이
- $K_w$: 커널의 너비
- $O_h$: 특성 맵의 높이
- $O_w$: 특성 맵의 너비
- $C_i$: 입력 데이터의 채널

다음은 3차원 텐서의 합성곱 연산을 보여줍니다.

<img src = "https://wikidocs.net/images/page/64066/conv16_final.png">

높이 $I_h$, 너비 $I_w$, 채널 $C_i$의 입력 데이터는 동일한 채널 수 $C_i$를 가지는 높이 $K_h$, 너비 $K_w$의 커널과 합성곱 연산을 하여 높이 $O_h$, 너비 $O_w$, 채널 1의 특성 맵을 얻습니다. 그런데 하나의 입력에 여러 개의 커널을 사용하는 합성곱 연산을 할 수도 있습니다. 

합성곱 연산에서 다수의 커널을 사용할 경우, 특성 맵의 크기가 어떻게 바뀌는지 봅시다. 다음은 $C_o$를 합성곱 연산에 사용하는 커널의 수라고 했을 때의 합성곱 연산 과정을 보여줍니다.

<img src = "https://wikidocs.net/images/page/64066/conv17_final_final.PNG">

합성곱 연산에서 다수의 커널을 사용할 경우, 사용한 커널 수는 합성곱 연산의 결과로 나오는 특성 맵의 채널 수가 됩니다.

이를 이해했다면 커널의 크기와 입력 데이터의 채널 수 $C_i$와 특성 맵(출력 데이터)의 채널 수 $C_o$가 주어졌을 때, 가중치 매개변수의 총 개수를 구할 수 있습니다. 가중치는 커널의 원소들이므로 하나의 커널의 하나의 채널은 $K_i \times K_o$개의 매개변수를 가지고 있습니다. 그런데 합성곱 연산을 하려면 커널은 입력 데이터의 채널 수와 동일한 채널 수를 가져야 합니다. 이에 따라 하나의 커널이 가지는 매개변수의 수는 $K_i \times K_o \times C_i$입니다. 그런데 이러한 커널이 총 $C_o$개가 있어야 하므로 가중치 매개변수의 총 수는 다음과 같습니다.

가중치 매개변수의 총 수: $K_i \times K_o \times C_i \times C_o$

## **9. 풀링(Pooling)**

일반적으로 합성곱 층(합성곱 연산 + 활성화 함수) 다음에는 풀링 층을 추가하는 것이 일반적입니다. 풀링 층에서는 특성 맵을 다운샘플링하여 특성 맵의 크기를 줄이는 풀링 연산이 이루어집니다. 풀링 연산에는 일반적으로 최대 풀링(max pooling)과 평균 풀링(average pooling)이 사용됩니다. 우선 최대 풀링을 통해 풀링 연산을 보겠습니다.

<img src = "https://wikidocs.net/images/page/62306/maxpooling.PNG">

풀링 연산에서도 합성곱 연산과 마찬가지로 커널과 스트라이드의 개념을 가집니다. 위의 그림은 스트라이드가 2일 때, 2 x 2 크기 커널로 맥스 풀링 연산과정입니다. 특성맵이 절반의 크기로 다운샘플링되는 것을 볼 수 있습니다. 맥스풀링은 커널과 겹치는 영역 안에서 최대값을 추출하는 방식으로 다운샘플링합니다.

평균 풀링은 최대값대신 평균값을 추출하는 연산이 됩니다. 풀링 연산은 커널가 스트라이드 개념이 존재한다는 점에서 합성곱 연산과 유사하지만 합성곱 연산과 차이점은 학습해야 할 가중치가 없으며 연산 후에 채널 수가 변하지 않는다는 점입니다. 

풀링을 사용하면, 특성맵의 크기가 줄어드므로 특성맵의 가중치 개수를 줄여줍니다.